# scrape Gutenberg

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request
import re
import time

In [80]:
authors_df = pd.read_csv("src/author_table.tsv", encoding="utf8", sep = "\t")

In [8]:
def get_html(website):
    try:
        with urllib.request.urlopen(website) as f:
            html_text = f.read()
    except urllib.error.URLError as e:
        print(e.reason)
    return html_text

In [20]:
list_of_genres = ["Roman",
                  #"Novellen",
                  "Novelle",
                  "Erzählung",]
                  #"Erzählungen",]

In [156]:
authors = []
genres = []
sources = []
titles = []
filenames = []

for website in list(authors_df.dropna(subset=["Gutenberg"])["Gutenberg"]):
    author = authors_df["Voller_Name"].loc[authors_df["Gutenberg"]==website].values[0]
    
    # print(author + "\t" + website)
    
    html_string = get_html(website)
    soup = BeautifulSoup(html_string, 'html.parser')
    
    for div in soup.find_all("div", {"class": "archived"}):
        for li in div.find_all("li"):
            if li.a:
                for genre in list_of_genres:
                    if genre in li.text:
                        authors.append(author)
                        genres.append(genre)
                        sources.append("https://www.projekt-gutenberg.org" + re.sub("\.\./\.\.", "", li.a.get("href")))
                        titles.append(li.a.text)
                        filenames.append(author.split()[-1] + "_" + re.sub("[\n\\ !:;,.'?/]", "", li.a.text) + "_gutenberg.txt")
                        
                        # print("\t" + li.a.text)
                        # print("\t" + genre)
                        # print("https://www.projekt-gutenberg.org" + re.sub("\.\./\.\.", "", li.a.get("href")))
                        # print("==========================")
        #time.sleep(2)

In [143]:
len(titles)

1168

In [157]:
df = pd.DataFrame()
df["title"]=titles
df["author"]=authors
df["filename"]=filenames
df["genre"]=genres
df["sources"]=sources

df.head()

,title,author,filename,genre,sources
0,Der Kinderball,Abraham Emanuel Fröhlich,Fröhlich_DerKinderball_gutenberg.txt,Erzählung,https://www.projekt-gutenberg.org/froehlia/kin...
1,Die Verschüttung im Hauenstein,Abraham Emanuel Fröhlich,Fröhlich_DieVerschüttungimHauenstein_gutenberg...,Erzählung,https://www.projekt-gutenberg.org/froehlia/hau...
2,Erzählungen,Achim von Arnim,Arnim_Erzählungen_gutenberg.txt,Erzählung,https://www.projekt-gutenberg.org/arnim/erzaeh...
3,verschiedene Novellen,Achim von Arnim,Arnim_verschiedeneNovellen_gutenberg.txt,Novelle,https://www.projekt-gutenberg.org/arnim/novell...
4,Bunte Steine,Adalbert Stifter,Stifter_BunteSteine_gutenberg.txt,Erzählung,https://www.projekt-gutenberg.org/stifter/bunt...


In [158]:
df.to_csv("metadata_gutenberg.csv", encoding="utf8", sep=";")

In [170]:
def get_chapter_list(url):
    chapters = []
    
    base_url = re.sub("(\w+-)?\w+.html", "", url)
    
    html_string = get_html(url)
    soup = BeautifulSoup(html_string, 'html.parser')
    
    for div in soup.find_all("div", {"class": "dropdown-content"}):
        for li in div.find_all("li"):
            chapters.append(base_url + li.a.get("href"))
    return list(set(chapters))

In [120]:
get_chapter_list("https://www.projekt-gutenberg.org/arnim/marino/marino.html")

['https://www.projekt-gutenberg.org/arnim/marino/titlepage.html',
 'https://www.projekt-gutenberg.org/arnim/marino/chap003.html',
 'https://www.projekt-gutenberg.org/arnim/marino/chap002.html',
 'https://www.projekt-gutenberg.org/arnim/marino/chap004.html']

In [171]:
i = 527

for text_website in list(df["sources"])[527:]:
    print(i)
    filename = df["filename"].loc[df["sources"]==text_website].values[0]
   
    chapters = get_chapter_list(text_website)
    
    text = ""
    
    if len(chapters)>0:
        for chapter in chapters:
            html_string = get_html(chapter)
            soup = BeautifulSoup(html_string, 'html.parser') 

            for p in soup.find_all("p")[1:]:
                text = text + p.text + "\n"
    else:
        html_string = get_html(text_website)
        soup = BeautifulSoup(html_string, 'html.parser') 

        for p in soup.find_all("p")[1:]:
            text = text + p.text + "\n"
    
    if len(text.split())<5:
        print(text_website)
    
    with open("gutenberg_files/" + filename, "w", encoding="utf8") as f:
        f.write(text)
    
    i=i+1

527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
599
600
601
602
603
604
605
606
607
608
609
610
611
612
613
614
615
616
617
618
619
620
621
622
623
624
625
626
627
628
629
630
631
632
633
634
635
636
637
638
639
640
641
642
643
644
645
646
647
648
649
650
651
652
653
654
655
656
657
658
659
660
661
662
663
664
665
666
667
668
669
670
671
672
673
674
675
676
677
678
679
680
681
682
683
684
685
686
687
688
689
690
691
692
693
694
695
696
697
698
699
700
701
702
703
704
705
706
707
708
709
710
711
712
713
714
715
716
717
718
719
720
721
722
723
724
725
726
727
728
729
730
731
732
733
734
735
736
737
738
739
740
741
742
743
744
745
746
747
748
749
750
751
752
753
754
755
756
757
758
759
760
761
762
763
764
765
766
767
768
769
770
771
772
773
774
775
776
